In [1]:
import json
import datetime
import hashlib

from typing import Any
from pprint import pprint
from dataclasses import dataclass, field, asdict

In [2]:
type SHA256 = str

In [3]:
@dataclass
class Block:
    index: int
    proof: int
    previous_hash: SHA256
    timestamp: str = field(default_factory=lambda: str(datetime.datetime.now()))

In [4]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_block(proof=1, previous_hash="0")

    def create_block(self, proof: int, previous_hash: str) -> Block:
        block = Block(
            index=len(self.chain) + 1, proof=proof, previous_hash=previous_hash
        )
        self.chain.append(block)
        return block

    def print_previous_block(self) -> dict:
        return self.chain[-1]

    def proof_of_work(self, previous_proof: int) -> int:
        new_proof = 1
        check_proof = False

        while check_proof is False:
            hash_operation = hashlib.sha256(
                str(new_proof**2 - previous_proof**2).encode()
            ).hexdigest()
            if hash_operation[:5] == "00000":
                check_proof = True
            else:
                new_proof += 1
        return new_proof

    def hash(self, block: Block):
        encoded_block = json.dumps(asdict(block), sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def chain_valid(self, chain: list[Block]) -> bool:
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block.previous_hash != self.hash(previous_block):
                return False

            previous_proof = previous_block.proof
            proof = block.proof
            hash_operation = hashlib.sha256(
                str(proof**2 - previous_proof**2).encode()
            ).hexdigest()
            if hash_operation[:5] != "00000":
                return False
            previous_block = block
            block_index += 1
        return True

    def __repr__(self):
        return f"Blockchain(len_chain={len(self.chain)})"


def mine_block(blockchain: Blockchain):
    previous_block = blockchain.print_previous_block()
    previous_proof = previous_block.proof
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)

    response = {
        "message": "A block is MINED",
        "index": block.index,
        "timestamp": block.timestamp,
        "proof": block.proof,
        "previous_hash": block.previous_hash,
    }

    return response


def display_chain(blockchain: Blockchain):
    response = {"chain": blockchain.chain, "length": len(blockchain.chain)}
    return response


def valid(blockchain: Blockchain):
    valid = blockchain.chain_valid(blockchain.chain)
    if valid:
        response = {"message": "The Blockchain is valid."}
    else:
        response = {"message": "The Blockchain is not valid."}
    return response

In [5]:
blockchain = Blockchain()

In [6]:
for i in range(3):
    print(f"Mine block: {i + 1}")
    pprint(mine_block(blockchain))
    print("-" * 100)

Mine block: 1
{'index': 2,
 'message': 'A block is MINED',
 'previous_hash': 'c80c2546a9e01582a98c98dfd848f70458d08ab8f59f2f41ee0b28c842ce609e',
 'proof': 632238,
 'timestamp': '2025-11-09 01:06:54.995127'}
----------------------------------------------------------------------------------------------------
Mine block: 2
{'index': 3,
 'message': 'A block is MINED',
 'previous_hash': 'd33614eb75490a4490400a16217d5a96c7059b92c76f41fc936b69f51d801ffd',
 'proof': 403091,
 'timestamp': '2025-11-09 01:06:56.735126'}
----------------------------------------------------------------------------------------------------
Mine block: 3
{'index': 4,
 'message': 'A block is MINED',
 'previous_hash': '1e4ee85e05bfbbbaf1e1735145cc0a1a931584ed52594d8d4f1c29e0add83c70',
 'proof': 714736,
 'timestamp': '2025-11-09 01:06:59.770167'}
----------------------------------------------------------------------------------------------------


In [7]:
blockchain.chain

[Block(index=1, proof=1, previous_hash='0', timestamp='2025-11-09 01:06:51.883230'),
 Block(index=2, proof=632238, previous_hash='c80c2546a9e01582a98c98dfd848f70458d08ab8f59f2f41ee0b28c842ce609e', timestamp='2025-11-09 01:06:54.995127'),
 Block(index=3, proof=403091, previous_hash='d33614eb75490a4490400a16217d5a96c7059b92c76f41fc936b69f51d801ffd', timestamp='2025-11-09 01:06:56.735126'),
 Block(index=4, proof=714736, previous_hash='1e4ee85e05bfbbbaf1e1735145cc0a1a931584ed52594d8d4f1c29e0add83c70', timestamp='2025-11-09 01:06:59.770167')]

In [8]:
display_chain(blockchain)

{'chain': [Block(index=1, proof=1, previous_hash='0', timestamp='2025-11-09 01:06:51.883230'),
  Block(index=2, proof=632238, previous_hash='c80c2546a9e01582a98c98dfd848f70458d08ab8f59f2f41ee0b28c842ce609e', timestamp='2025-11-09 01:06:54.995127'),
  Block(index=3, proof=403091, previous_hash='d33614eb75490a4490400a16217d5a96c7059b92c76f41fc936b69f51d801ffd', timestamp='2025-11-09 01:06:56.735126'),
  Block(index=4, proof=714736, previous_hash='1e4ee85e05bfbbbaf1e1735145cc0a1a931584ed52594d8d4f1c29e0add83c70', timestamp='2025-11-09 01:06:59.770167')],
 'length': 4}

In [9]:
valid(blockchain)

{'message': 'The Blockchain is valid.'}